#### Code for lstm network development. data preparation for lstm can be found in preprocess/seqdata.ipynb

In [13]:
import pandas as pd
import numpy as np
from math import ceil
import time
import glob,os
import matplotlib.pyplot as plt
import pickle
import pdb
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [ ]:
#load data
with open('/home/arash/ProjectVR/cleaneddata/seqdata', 'rb') as f:
    allseqdata = pickle.load(f)

inputseq=allseqdata[0]
ypadded=allseqdata[1]

In [ ]:
#####not use scaled measures for now
scaler = MinMaxScaler(feature_range=(0, 1))
Xscaled = scaler.fit_transform(Xpadded)

yscaled = scaler.fit_transform(ypadded)

o1scaled = scaler.fit_transform(o1padded)
o2scaled = scaler.fit_transform(o2padded)
o3scaled = scaler.fit_transform(o3padded)
distscaled = scaler.fit_transform(distpadded)

In [7]:
#check if data is float:
print(ypadded.dtype,inputseq.dtype)

float32 float32


In [8]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = inputseq.shape[2]
n_steps_in=inputseq.shape[1]
n_steps_out=ypadded.shape[1]

In [9]:
# seperate validation data and test set
import random
tst=0.2                 #% if test data
val=0.2                 #% if valid data       
tstsize = int(np.floor(len(inputseq) * tst))
valsize = int(np.floor(len(inputseq) * val))


tstlabel=[]
for i in range(tstsize):
    temp = random.randint(0,inputseq.shape[0]+1)
    tstlabel.append(temp)

Xtest=inputseq[tstlabel]
ytest=ypadded[tstlabel]

In [10]:
trnlabel=[i for i in range(inputseq.shape[0]) if i not in tstlabel]
trnsize=len(trnlabel)

Xtrain=inputseq[trnlabel]          #Excluding test set
ytrain=ypadded[trnlabel]

In [11]:
vallabel=[]
for i in range(valsize):
    temp = trnlabel[random.randint(0,trnsize+1)]
    vallabel.append(temp)

Xval=inputseq[vallabel]
yval=ypadded[vallabel]

trainlabel=[i for i in range(Xtrain.shape[0]) if i not in [vallabel]]
Xtrain=Xtrain[trainlabel]
ytrain=ytrain[trainlabel]

In [ ]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
history=model.fit(Xtrain, ytrain,
                  epochs=200, verbose=1,batch_size=72,
                  validation_data=(Xval, yval))
                  



# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
#pyplot.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 2711 samples, validate on 661 samples
Epoch 1/200
2711/2711 [==============================] - 3s 1ms/step - loss: 25923796716757.2109 - val_loss: 504823.5529
Epoch 2/200
2711/2711 [==============================] - 2s 901us/step - loss: 480971.7508 - val_loss: 74022.1840
Epoch 3/200
2711/2711 [==============================] - 2s 911us/step - loss: 1534724.0731 - val_loss: 114989.2299
Epoch 4/200
2711/2711 [==============================] - 2s 905us/step - loss: 169372.2583 - val_loss: 71134.1667
Epoch 5/200
2711/2711 [==============================] - 2s 903us/step - loss: 33853098.4521 - val_loss: 66567.2036
Epoch 6/200
2711/2711 [==============================] - 2s 901us/step - loss: 72026.6451 - val_loss: 76709.9956
Epoch 7/200
2711/2711 [==============================] - 2s 906us/step - loss: 315485.8847 - val_loss: 470947.5257
Epoch 8/200
2711/2711 [=

Epoch 67/200
2711/2711 [==============================] - 2s 892us/step - loss: 163796.4023 - val_loss: 88546.9951
Epoch 68/200
2711/2711 [==============================] - 2s 899us/step - loss: 822744.4253 - val_loss: 128658.0589
Epoch 69/200
2711/2711 [==============================] - 2s 903us/step - loss: 1109898.5213 - val_loss: 79803.3288
Epoch 70/200
2711/2711 [==============================] - 2s 892us/step - loss: 309665.1327 - val_loss: 77363.6415
Epoch 71/200
2711/2711 [==============================] - 2s 892us/step - loss: 128957.4975 - val_loss: 71570.1546
Epoch 72/200
2711/2711 [==============================] - 2s 897us/step - loss: 101432.6817 - val_loss: 72006.4447
Epoch 73/200
2711/2711 [==============================] - 2s 896us/step - loss: 75294.5448 - val_loss: 70134.4288
Epoch 74/200
2711/2711 [==============================] - 2s 896us/step - loss: 70338.3158 - val_loss: 69917.5249
Epoch 75/200
2711/2711 [==============================] - 2s 907us/step - loss: 

In [ ]:

# make a prediction
yhat = model.predict(Xval)